In [1]:
# pull ACMI-linked Wikidata records to check multiple type declaration.

import numpy
import pandas
import pathlib
import pydash
import requests

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    data_frame = pandas.DataFrame.from_dict(results)
    for column in data_frame.columns:
        data_frame[column] = data_frame.apply(value_extract, column=column, axis=1)
    
    return data_frame

wikidata_query = ''' 
    select distinct ?wikidata_id ?wikidata_typeLabel where { 
        ?wikidata_id wdt:P7003 ?acmi_id .
        ?wikidata_id wdt:P31 ?wikidata_type .
        service wikibase:label { bd:serviceParam wikibase:language "en". }
        } '''
wikidata_type = sparql_query(wikidata_query, 'https://query.wikidata.org/sparql')

wikidata_type['wikidata_id'] = wikidata_type['wikidata_id'].str.split('/').str[-1]
wikidata_type = wikidata_type.pivot_table(index=['wikidata_id'], aggfunc=lambda x: ','.join(x)).reset_index()
wikidata_type = wikidata_type.loc[wikidata_type.wikidata_typeLabel.str.contains(',', na=False)]

print(len(wikidata_type))
wikidata_type.sample(20)

150


,wikidata_id,wikidata_typeLabel
9196,Q7732733,"television film,television series episode"
312,Q110944326,"film series,miniseries,documentary film"
8031,Q632532,"television film,miniseries"
9131,Q767850,"animated television series,film"
9913,Q947098,"feature film,film adaptation"
3191,Q205447,"remake,film"
9031,Q751921,"film,cult film"
7951,Q623051,"film,cult film"
9247,Q7751655,"television series,film"
5036,Q3203035,"silent film,short film"
